# Labs Project - CourtVision
*An update by Benjamin de Charmoy*



## Outline
* Labs Projects
* What is CourtVision?
* Pieces of the puzzle
* Following the path
* Where are we - where to next?
* Questions / Ideas

### Labs Projects
* Thinkst Applied Research - Labs opened up a space for Thinksters to pursue research
* CourtVision is the first such project
    - I spoke to Jacob and we kicked around a few ideas. I am super interested in computer vision / signal processing (Magic!)
    - Some initial noodling lead us to focus on single camera system
    - A few iterations of the proposal
    - Chats with Marco around when to do it and navigating existing engineering work
    - Then we had lift off!
* Thanks @thinkst, @mh, @mls and @jacob for creating this space
    - https://phabricator.think.st/w/labs/welcome_call/  
    - Read about it here [Labs Research Program](https://docs.google.com/document/d/1XWZX3s6kzoOOr7oewH9W16ftyJlf7JSVd_kHTyBEWvs/edit)
    - [#AllHands announcement](https://drive.google.com/file/d/1SDAliPgejyoLKGjF8Mxn_CuJ8LR9NuSN/view?pli=1)

* You and your research - Labs slide Propose, Motivate, Do -(we can chat if I should/n't add this here)
* What is CourtVision - applied to Padel (clip and highlevel description)
* Pieces of the puzzle:
    - Image plane detections (detect [x, y] - Players, Ball, Court markings)
    - Relate image plan to real world
    - Track Players and Ball to better estimate real world potitions
    - Visualize game play and see what players could learn / discover
* Following the path:
    - Here I have sides on approaches and some details / code explaining
    - Annatomy of the ML process showing intermediate results - gotchas, oversights

* Where are we - where to next:
    - Nice visuals of what one can currently do.

* Questions / Ideas
    - 
    https://stackoverflow.com/questions/37743144/how-to-increase-the-page-width-in-ipython-notebook-jupyter-slides

## What is CourtVision?


* CourtVision is a research project into extracting useful information from videos of court-based sports.


* Looking primarily at Padel
<img  src="padel-omg-omg-omg.gif" style=" width: 200%">




## What is CourtVision?

* CourtVision goals
    - A dataset curated from live streamed events (we aim to share)
    - A computer vision package and technical write up
    - Evaluation of the efficacy of a single camera system
    - Visualizations to evaluate the usefulness of CourtVision
* We are currently ▓▓▓▓▓▓▓▓▓▓▓▓▓░░░░░░░ 65% of the way through



# Pieces of the puzzle

**Pieces of the puzzle**
<!-- 
<p align="center" width="100%" height="100%">
    <img  src="the_puzzle.png">
</p> -->
<!-- ![](the_puzzle_small.png) -->
<img src="the_puzzle.png" alt="Drawing" style=" width: 200% "/>
 


**Pieces of the puzzle**
<img src="the_puzzle_top.png" alt="Drawing" style=" width: 200%"/>

**Pieces of the puzzle**
<img src="the_puzzle_bottom.png" alt="Drawing" style=" width: 200%"/>

## Pieces of the puzzle
* Data generating process
    - Events unfold in the world - games are played
    - A camera with unknown internal characteristics captues these events
* Data undestanding process
    - Detections are made in the "Image plane" 
    -  Camera **intrisics** are computed using knowledge about the scene
    - Camera **extrinsics** are computed uisng knowledge about the scene
    - A **one-way** mapping from "World coordinate frame" to "image plane"
    - A homogrphy (transform - plane to plane) between image and world plane
    - **BUT** we only have image plane detections how can we establish world coordinates of players and ball?
    - Visualize the output in a way that is useful to the Players.
        + Seeing Spaces is a great talk on this - bret victor



# Following the path

<p align="center" width="100%">
    <img  src="tinker_eng_sci.png">
</p>
  

### Following the path
<img  src="the_path.png" style = "width: 200%">

### Following the path
* Fetching data - downloaded hours of Padel games
* Extracted frames and audio
    - This was not very robust (cycling back to use gesture recognition)
<img src="audio.png" style = "width: 70%" />
    


### Following the path
* Ball and Player detector
    


In [24]:
%%time
from courtvision.models import get_fasterrcnn_ball_detection_model
from courtvision.vis import load_timg, draw_rect
import torch

ball_detector = get_fasterrcnn_ball_detection_model(
    "../../models/ball_detector/fasterrcnn_resnet50_fpn_project-1-at-2023-05-05-03-25-2cd00928.pt"
)
ball_detector.eval()
preds = ball_detector(
    load_timg("../../data/frames/curated_001/curated_001_frame_0021.png")
)[0]
print(preds["boxes"][:2], preds["labels"][:2], preds["scores"][:2])
preds.keys()

tensor([[ 881.7243,  369.5026,  895.3730,  379.6957],
        [1139.6188,  367.9913, 1149.7804,  380.5721]],
       grad_fn=<SliceBackward0>) tensor([1, 1]) tensor([0.4254, 0.3532], grad_fn=<SliceBackward0>)
CPU times: user 1.84 s, sys: 2.23 s, total: 4.07 s
Wall time: 1.93 s


dict_keys(['boxes', 'labels', 'scores'])

In [21]:
ball_detector

FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       

### Following the path
* Compute scene geometry
<img src="court_labelling.png" style="width: 100%"/>


### Following the path
* Combine detector and geometry - a unified view
<img  src="PlaneTracking2.gif" style = "width: 100%">



### Following the path
* The detour! 
* Combine tracker and geometry
    - Rerun visual (demo)

<img src = "pose_box.png" style = "height: 100%">          



# Where are we? 
* CourtVision goals
    - A dataset curated from live streamed events  ▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓░░ 90%
    - A computer vision package and technical write up ▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓░░░░░ 75%
    - Evaluation of the efficacy of a single camera system ▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓░░░░ 80%
    - Visualizations to evaluate the usefulness of CourtVision ▓▓▓▓▓▓░░░░░░░░░░░░░░ 30%
    


# Where to next?
* Improve the Tracker and one last loop of the Path
* Generate more player centric informative visualizations
* Run on a mobile device
* Relax the constraint of a stationary camera


# Questions / Ideas


# See you on court 